<a href="https://colab.research.google.com/github/Ihsan1331/Space_Titanic/blob/main/Space_Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opendatasets
!pip install joblib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 1. Dataset and Package

---

In this section we will upload the necessary package and dataset

In [ ]:
import opendatasets as od
import joblib
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from joblib import dump, load
od.download('https://www.kaggle.com/competitions/spaceship-titanic')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: ahmadihsanfarhani
Your Kaggle Key: ··········


100%|██████████| 299k/299k [00:00<00:00, 81.8MB/s]


Extracting archive ./spaceship-titanic/spaceship-titanic.zip to ./spaceship-titanic


In [ ]:
data = pd.read_csv(r'/content/spaceship-titanic/train.csv')
test = pd.read_csv(r'/content/spaceship-titanic/test.csv')

Save the "train" data into data variable and test data into "test" variable.

# 2. Preprocess the Data

In [ ]:
print('Shape of dataset = ', data.shape)
print('All Features and type of dataset =')
data.dtypes

Shape of dataset =  (8693, 14)
All Features and type of dataset =


PassengerId      object
HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported        bool
dtype: object

There are 8693 sample and 14 features

In [ ]:
data.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


## 2.1 Checking Null or NaN in Dataset

In [ ]:
data.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

All of Features Contain NaN or Null

## 2.2 Dropping Name and PassengerID Features 

In [ ]:
dropped_data = data.drop(['PassengerId', 'Name'], axis = 1)
dropped_data = dropped_data.dropna()

In [ ]:
dropped_data.isnull().sum()

HomePlanet      0
CryoSleep       0
Cabin           0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Transported     0
dtype: int64

## 2.3 Creating Deck features from Cabin Features

In [ ]:
cabin = []
for i in dropped_data['Cabin']:
  cabin.append(i[0])

In [ ]:
dropped_data['Cabin'] = cabin

In [ ]:
dropped_data[['Cabin', 'Transported']].groupby(['Cabin'], as_index=False).mean().sort_values(by='Transported', ascending=False)

,Cabin,Transported
1,B,0.730109
2,C,0.677796
6,G,0.522041
0,A,0.483568
5,F,0.434330
3,D,0.425197
4,E,0.354286
7,T,0.333333


Cabin features affect the likelihood a person transported or not. person in cabin B has the biggest likelihood to transported while T has the lowest likelihood to transported

## 2.4 Check if feature Homeplanet affect person likelihood to be transported

In [ ]:
dropped_data[['HomePlanet', 'Transported']].groupby(['HomePlanet'], as_index=False).mean().sort_values(by='Transported', ascending=False)

,HomePlanet,Transported
1,Europa,0.658095
2,Mars,0.515390
0,Earth,0.425328


Person from Europa has the biggest likelihood to be transported

## 2.5 Check if feature CryoSleep affect person likelihood to be transported

In [ ]:
dropped_data[['CryoSleep', 'Transported']].groupby(['CryoSleep'], as_index=False).mean().sort_values(by='Transported', ascending=False)

,CryoSleep,Transported
1,True,0.815966
0,False,0.332801


Person undergoing cryosleep has the biggest likelihood to be transported

## 2.6 Check if feature VIP affect person likelihood to be transported

In [ ]:
dropped_data[['VIP', 'Transported']].groupby(['VIP'], as_index=False).mean().sort_values(by='Transported', ascending=False)

,VIP,Transported
0,False,0.505909
1,True,0.378049


VIP has the biggest likelihood to be transported

## 2.7 Check if feature Destination affect person likelihood to be transported

In [ ]:
dropped_data[['Destination', 'Transported']].groupby(['Destination'], as_index=False).mean().sort_values(by='Transported', ascending=False)

,Destination,Transported
0,55 Cancri e,0.617892
1,PSO J318.5-22,0.504687
2,TRAPPIST-1e,0.467108


## 2.8 Applying One Hot Encoder to Data

In [ ]:
new_data = pd.get_dummies(dropped_data, drop_first = True)

In [ ]:
new_data.head()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_True,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_True
0,39.0,0.0,0.0,0.0,0.0,0.0,False,1,0,0,1,0,0,0,0,0,0,0,1,0
1,24.0,109.0,9.0,25.0,549.0,44.0,True,0,0,0,0,0,0,0,1,0,0,0,1,0
2,58.0,43.0,3576.0,0.0,6715.0,49.0,False,1,0,0,0,0,0,0,0,0,0,0,1,1
3,33.0,0.0,1283.0,371.0,3329.0,193.0,False,1,0,0,0,0,0,0,0,0,0,0,1,0
4,16.0,303.0,70.0,151.0,565.0,2.0,True,0,0,0,0,0,0,0,1,0,0,0,1,0


## 2.9 Normalize RoomService, FoodCourt, ShoppingMall, Spa, VRDeck and Age features so that all features in same range

In [ ]:
scaler = MinMaxScaler()
new_data[['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']] = scaler.fit_transform(new_data[['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']])

## 2.10 Split Train data into train and test

In [ ]:
y = new_data['Transported']
split_data = new_data.drop(['Transported'], axis = 1)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(split_data, y, test_size = 0.2, stratify = y) 

# 3. Fitting Model

In [ ]:
new_data.head()

## 3.1 Choosing best hyperparameter for SVC

In [ ]:
parameter_svc = {'C' : [0.5,1,2], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid']}

GSCV_svc = GridSearchCV(SVC(), 
                       param_grid = parameter_svc, 
                       scoring = 'accuracy',
                       cv = 5,
                       return_train_score = True
)
resultssvc = GSCV_svc.fit(x_train, y_train)
results_svc_df = pd.DataFrame.from_dict(resultssvc.cv_results_)
display(results_svc_df[['params','mean_test_score','mean_train_score']])
print('parameter terbaik = ', GSCV_svc.best_params_)

## 3.2  Choosing best hyperparameter for Random Forest

In [ ]:
parameter_rf = {'n_estimators' : [50, 100, 200], 'criterion' : ['gini', 'entropy']}

GSCV_rf = GridSearchCV(RandomForestClassifier(), 
                       param_grid = parameter_rf, 
                       scoring = 'accuracy',
                       cv = 5,
                       return_train_score = True
)
resultsrf = GSCV_rf.fit(x_train, y_train)
results_rf_df = pd.DataFrame.from_dict(resultsrf.cv_results_)
display(results_rf_df[['params','mean_test_score','mean_train_score']])
print('parameter terbaik = ', GSCV_rf.best_params_)

## 3.3 Choosing best hyperparameter for Logistic Regression

In [ ]:
parameter_lr = {'C' : [0.5,1,2]}

GSCV_lr = GridSearchCV(LogisticRegression(), 
                       param_grid = parameter_lr, 
                       scoring = 'accuracy',
                       cv = 5,
                       return_train_score = True
)
resultslr = GSCV_lr.fit(x_train, y_train)
results_lr_df = pd.DataFrame.from_dict(resultslr.cv_results_)
display(results_lr_df[['params','mean_test_score','mean_train_score']])
print('parameter terbaik = ', GSCV_lr.best_params_)

##3.4 Creating Voting Classifier

In [ ]:
voting = VotingClassifier(estimators = [('rf', RandomForestClassifier(criterion= 'gini', n_estimators= 200)),
                                        ('lr', LogisticRegression(C= 2)),
                                        ('svm', SVC(C= 2, kernel = 'rbf'))],
                                         voting = 'hard')
voting.fit(x_train, y_train)

## 3.5 Evaluate Prediction

In [ ]:
y_predict = voting.predict(x_test)

In [ ]:
confusion_matrix(y_test, y_predict)

In [ ]:
accuracy_score(y_test, y_predict)

# 4. Testing Model

In [ ]:
test_data = test
test_data = test_data.drop(['PassengerId', 'Name'], axis = 1)
test.isnull().sum()

In [ ]:
test_data['HomePlanet'] = test_data['HomePlanet'].fillna(test_data['HomePlanet'].mode()[0])
test_data['CryoSleep'] = test_data['CryoSleep'].fillna(test_data['CryoSleep'].mode()[0])
test_data['Cabin'] = test_data['Cabin'].fillna(test_data['Cabin'].mode()[0])
test_data['Destination'] = test_data['Destination'].fillna(test_data['Destination'].mode()[0])
test_data['Age'] = test_data['Age'].fillna(test_data['Age'].mean())
test_data['VIP'] = test_data['VIP'].fillna(test_data['VIP'].mode()[0])
test_data['RoomService'] = test_data['RoomService'].fillna(test_data['RoomService'].mean())
test_data['FoodCourt'] = test_data['FoodCourt'].fillna(test_data['FoodCourt'].mean())
test_data['ShoppingMall'] = test_data['ShoppingMall'].fillna(test_data['ShoppingMall'].mean())
test_data['Spa'] = test_data['Spa'].fillna(test_data['Spa'].mean())
test_data['VRDeck'] = test_data['VRDeck'].fillna(test_data['VRDeck'].mean())
test_data.isnull().sum()

In [ ]:
cabin_test = []
for i in range (0, len(test_data)):
  cabin_test.append(test_data['Cabin'][i][0])
test_data['Cabin'] = cabin_test

In [ ]:
test_data.head()

In [ ]:
test_data[['CryoSleep', 'VIP']] = test_data[['CryoSleep', 'VIP']].astype('object')
test_data = pd.get_dummies(test_data, drop_first = True)
test_data[['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']] = scaler.fit_transform(test_data[['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']])

In [ ]:
test_data.head()

In [ ]:
y_test_predict = voting.predict(test_data)

In [ ]:
pd_y_test = pd.DataFrame(y_test_predict, columns = ['Transported'])
pd_y_type = pd_y_test.astype('object')
pd_y_test.head()

In [ ]:
pd_id = pd.DataFrame(test['PassengerId'])
pd_id.head()

In [ ]:
pd.concat([pd_id, pd_y_test], axis = 1).to_csv('Space_Titanic.csv', index = False)

# 5. Saving Model

In [ ]:
dump(voting, 'Space_titanic.joblib') 

In [ ]:
saved_model = load('Space_titanic.joblib')
saved_model.predict(x_test) == y_predict